# Problem 1

In [1]:
%matplotlib inline

import os

# Generate images with condition labels
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn

from tqdm import trange
import numpy as np
import time as t

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 1
batch_size = 200

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Put your code here
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False) 

In [4]:
class GeneratioNet(torch.nn.Module):
    
    def __init__(self):
        
        super(GeneratioNet, self).__init__()
        
        self.Decoder = nn.Sequential(
            nn.Conv2d(1, 256, 2, stride=2, padding=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            
            nn.ConvTranspose2d(256, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Upsample(scale_factor=2),
            
            nn.ConvTranspose2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Upsample(scale_factor=2),
            
            nn.ConvTranspose2d(64, 32, 3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            
            nn.ConvTranspose2d(32, 1, 3, stride=1, padding=1),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )


    def forward(self, x):
        """The forward function should return batch of images."""
        
        x = x.view(batch_size, 1, 10, 10)
        x = self.Decoder(x)
        
        return x

In [5]:
class DiscrimiNet(torch.nn.Module):
    
    def __init__(self):
        super(DiscrimiNet, self).__init__()

        self.Encoder = nn.Sequential(
                nn.Conv2d(1, 64, 3, stride=1, padding=1),
                nn.BatchNorm2d(64),
                nn.LeakyReLU(),
                nn.MaxPool2d(2),
                nn.Dropout(0.5),
            
                nn.Conv2d(64, 128, 3, stride=1, padding=1),
                nn.BatchNorm2d(128),
                nn.LeakyReLU(),
                nn.MaxPool2d(2),
                nn.Dropout(0.5),
            
                nn.Conv2d(128, 256, 3, stride=2, padding=1),
                nn.BatchNorm2d(256),
                nn.LeakyReLU(),
                nn.Dropout(0.5),
            
                nn.Conv2d(256, 512, 3, stride=1, padding=1),
                nn.BatchNorm2d(512),
                nn.LeakyReLU(),
                nn.Dropout(0.5),
            )
        
        self.Regression = nn.Linear(512*4*4, 1)
        
    def forward(self, x):
        """The forward function should return the logits."""
        
        x = self.Encoder(x)
        x = x.view(x.size(0), -1)
        x = self.Regression(x)
        
        return x

In [6]:
class DCGANet(object):
    
    def __init__(self, epochs, batch_size):
        
        ##### ---- YOUR CODE HERE ---- #####
        self.G = GeneratioNet()
        self.D = DiscrimiNet()
        self.loss = nn.MSELoss()
        ##### ----                ---- #####

        self.d_optimizer = torch.optim.Adam(self.D.parameters(), lr=0.0002, betas=(0.5, 0.999))
        self.g_optimizer = torch.optim.Adam(self.G.parameters(), lr=0.0002, betas=(0.5, 0.999))

        self.epochs = epochs
        self.batch_size = batch_size

        self.number_of_images = 10
        
    def train(self, train_loader):
        
        disc_loss = []
        genr_loss = []
        
        generator_iter = 0
        
        for epoch in trange(self.epochs):

            for i, (images, _) in enumerate(train_loader):
                print(f"Epoch {epoch+1}: Batch {i+1} of {len(train_loader)}")
                
                # Step 1: Train discriminator
                z = torch.rand((self.batch_size, 100, 1, 1))
                
                real_labels = torch.ones(self.batch_size)
                fake_labels = torch.zeros(self.batch_size)

                images, z = images.to(device), z.to(device)
                real_labels, fake_labels = real_labels.to(device), fake_labels.to(device)

                # Compute the BCE Loss using real images
                real_logits = self.D(images)
                real_logits = torch.squeeze(real_logits)
                d_loss_real = self.loss(real_logits, real_labels)

                # Compute the BCE Loss using fake images
                print("Generating images...")
                fake_images = self.G(z)
                print("Discriminating images...")
                fake_logits = self.D(fake_images)
                fake_logits = torch.squeeze(fake_logits)
                d_loss_fake = self.loss(fake_logits, fake_labels)

                # Optimize discriminator
                d_loss = d_loss_real + d_loss_fake
                self.D.zero_grad()
                d_loss.backward()
                self.d_optimizer.step()

                # Step 2: Train Generator
                z = torch.randn(self.batch_size, 100, 1, 1).to(device)
                
                fake_images = self.G(z)
                fake_logits = self.D(fake_images)
                fake_logits = torch.squeeze(fake_logits)
                g_loss = self.loss(fake_logits, real_labels)

                self.D.zero_grad()
                self.G.zero_grad()
                g_loss.backward()
                self.g_optimizer.step()
                generator_iter += 1

                disc_loss.append(d_loss.item())
                genr_loss.append(g_loss.item())

        return disc_loss, genr_loss

    def generate_img(self, z, number_of_images):
        samples = self.G(z).data.cpu().numpy()[:number_of_images]
        generated_images = []
        for sample in samples:
            generated_images.append(sample.reshape(28, 28))
        return generated_images

In [ ]:
# set number_of_images, z
number_of_images = 1
z = torch.randn(batch_size, 100)

# set up model
# run training
model = DCGANet(epochs, batch_size)
disc_loss, genr_loss = model.train(train_loader)

# Problem 2

In [83]:
import torch
import torch.nn as nn
import nltk
from nltk.corpus import stopwords
import numpy as np
from tqdm import trange
from bs4 import BeautifulSoup
import re,string,unicodedata, copy
import random
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
device = 'cpu'

nltk.download('punkt')

#download corpus
nltk.download('gutenberg')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /home/jovyan/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
#Removing all the noise
def denoise_text(text):
    #remove html strips
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    #remove \n
    text = re.sub('\n', ' ', text)
    #remove square brackets
    text = re.sub('\[[^]]*\]', '', text)
    #replace punctuation with space
    text = re.sub(r'[,.;@#?!&$\-]+\ *', ' ', text, flags=re.VERBOSE)
    #remove special characters
    text=re.sub(r'[^a-zA-z0-9\s]', '', text)
    #remove extra spaces
    text = re.sub(' +', ' ', text)
    return text.lower().strip()

In [43]:
# load dataset
corpus_raw = nltk.corpus.gutenberg.raw('austen-sense.txt')

# clean dataset
corpus_raw = denoise_text(corpus_raw)
words = corpus_raw.split(' ')

# tokenize
corpus_t = nltk.word_tokenize(corpus_raw)

# convert to one-hot encoding
unique_words = np.unique(corpus_t)
d = {word: i for i, word in enumerate(unique_words)}
corpus_n = torch.Tensor([d[word] for word in corpus_t])
corpus = F.one_hot(corpus_n.to(torch.int64))

# generate 6-grams
# split into 5-grams and targets
grams_pre = torch.stack(
    [corpus[i:i+6,:] for i in range(len(corpus)-6+1)], 
    dim=-1).permute(2,0,1)
grams = grams_pre[:,:5,:]
targets = grams_pre[:,5,:]

# pad dataset to N=100
#grams = torch.cat((grams,
#                     torch.zeros(len(grams), 95, len(unique_words)),
#                     dim=1)
# too big ???

In [44]:
# build data loader

gram_set = TensorDataset(grams, targets)
dl = DataLoader(gram_set, batch_size=64)

In [117]:
# build model

class LSTMGenerator(nn.Module):
    def __init__(self, in_dim, h_dim, n_classes, n_layers):
        super(LSTMGenerator, self).__init__()
        self.in_dim = in_dim
        self.h_dim = h_dim
        self.n_classes = n_classes
        self.n_layers = n_layers
        
        self.lstm = torch.nn.LSTM(input_size = self.in_dim, 
                                  hidden_size = self.h_dim, 
                                  num_layers = self.n_layers, 
                                  batch_first=True)
        self.classifier = nn.Linear(in_features=self.h_dim, 
                                    out_features=self.n_classes)

    def forward(self, x):
        # initialize H and C
        h0 = torch.zeros((self.n_layers, len(x), self.h_dim), dtype=x.dtype).to(device)
        c0 = torch.zeros((self.n_layers, len(x), self.h_dim), dtype=x.dtype).to(device)
        
        m, _ = self.lstm(x, (h0, c0))
        
        logits = self.classifier(m[:, -1, :])
        return logits

In [81]:
_, (grams, targets) = next(enumerate(dl))

In [122]:
in_dim = len(unique_words)
h_dim = 1000
n_classes = len(unique_words)
n_layers = 1
grams = grams.long()

out = LSTMGenerator(in_dim, h_dim, n_classes, n_layers)
out2 = out(grams.float())